In [3]:
import numpy as np
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install -qq langdetect
from langdetect import detect
import string
!pip install -qq contractions
!pip install -qq demoji
import demoji
demoji.download_codes()
!pip install -qq datasets

<ipython-input-1-28a5514cb77a>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [4]:
from datasets import load_dataset

dataset = load_dataset("offenseval_dravidian", "kannada")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6217 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/777 [00:00<?, ? examples/s]

In [5]:
df_train = pd.DataFrame(dataset["train"])
label_mapping = {
    0: 'Not_offensive',
    1: 'Offensive',
    2: 'Offensive',
    3: 'Offensive',
    4: 'Offensive',
    5: 'not-Kannada',
}
df_train['label'].replace(label_mapping, inplace=True)

label_mapping1 = {
    'Not_offensive':0,
    'Offensive':1,
    'not-Kannada':2,
}
df_train['label'].replace(label_mapping1, inplace=True)

print(df_train.info())
label_counts = df_train['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6217 entries, 0 to 6216
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6217 non-null   object
 1   label   6217 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 97.3+ KB
None
0    3544
2    1522
1    1151
Name: label, dtype: int64


In [6]:
df_train.head(20)

,text,label
0,Tik tok alli jagala madtidralla adra baggenu o...,0
1,Anyone from kerala here,2
2,Movie rerelease madi plss,0
3,Amazon prime alli bittidira....yella manele no...,0
4,Guru sure news nanu tik tok dawn lod madeda ya...,0
5,ಸುದೀಪ್ ಸರ್ ಅಂಡ್ ದರ್ಶನ್ ಸರ್ ಅವರಿಗೆ ಇರೋ ಫ್ಯಾನ್ಸ್...,0
6,Ade old same story,1
7,Superb rakshit sir sobg superb macking,2
8,Hai Neel (prithvi) wonderful movie dia,2
9,ಏನ್ ಗುರು ನಿನ್ನ ಚರಿತ್ರೆ ಸೃಷ್ಟಿಸುವ ಅವತಾರ ತುಂಬಾ ಚ...,0


In [7]:
import sys
sys.path.append("/content/drive/MyDrive/Capstone/Translator/")
from google_trans_new import google_translator

In [8]:
translator = google_translator()
for index, row in df_train.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'kn': # Text is Kannada
            for key, value in found.items():
                emojiEng = value
                # Translate to Kannada
                transText = translator.translate(emojiEng, lang_tgt='kn', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_train.at[index, 'text'] = text

In [9]:
df_train.head(30)

,text,label
0,Tik tok alli jagala madtidralla adra baggenu o...,0
1,Anyone from kerala here,2
2,Movie rerelease madi plss,0
3,Amazon prime alli bittidira....yella manele no...,0
4,Guru sure news nanu tik tok dawn lod madeda ya...,0
5,ಸುದೀಪ್ ಸರ್ ಅಂಡ್ ದರ್ಶನ್ ಸರ್ ಅವರಿಗೆ ಇರೋ ಫ್ಯಾನ್ಸ್...,0
6,Ade old same story,1
7,Superb rakshit sir sobg superb macking,2
8,Hai Neel (prithvi) wonderful movie dia,2
9,ಏನ್ ಗುರು ನಿನ್ನ ಚರಿತ್ರೆ ಸೃಷ್ಟಿಸುವ ಅವತಾರ ತುಂಬಾ ಚ...,0


In [ ]:
for index, row in df_train.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [10]:
import contractions
# contracted text
for index, row in df_train.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_train.at[index, 'text'] = expanded_text


for index, row in df_train.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_train.at[index, 'text'] = Sentence.lower()

In [11]:
df_train.head(30)

,text,label
0,tik tok alli jagala madtidralla adra baggenu o...,0
1,anyone from kerala here,2
2,movie rerelease madi plss,0
3,amazon prime alli bittidirayella manele nodtare,0
4,guru sure news nanu tik tok dawn lod madeda ya...,0
5,ಸುದೀಪ್ ಸರ್ ಅಂಡ್ ದರ್ಶನ್ ಸರ್ ಅವರಿಗೆ ಇರೋ ಫ್ಯಾನ್ಸ್...,0
6,ade old same story,1
7,superb rakshit sir sobg superb macking,2
8,hai neel prithvi wonderful movie dia,2
9,ಏನ್ ಗುರು ನಿನ್ನ ಚರಿತ್ರೆ ಸೃಷ್ಟಿಸುವ ಅವತಾರ ತುಂಬಾ ಚ...,0


In [15]:
!pip install -qq indic-nlp-library
!pip install -qq tqdm
from indicnlp import common
from indicnlp import loader
from indicnlp.script import indic_scripts
from indicnlp.tokenize import indic_tokenize
from indicnlp.morph import unsupervised_morph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 16.0 MB/s eta 0:00:00


In [16]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
INDIC_NLP_RESOURCES = r"/content/indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 29.99 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [12]:
!pip install -qq nltk

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
kannada_stopwords = [
    'ಮತ್ತು', 'ಈ', 'ಒಂದು', 'ರಲ್ಲಿ', 'ಹಾಗೂ', 'ಎಂದು', 'ಅಥವಾ', 'ಇದು', 'ರ', 'ಅವರು', 'ಎಂಬ',
    'ಮೇಲೆ', 'ಅವರ', 'ತನ್ನ', 'ಆದರೆ', 'ತಮ್ಮ', 'ನಂತರ', 'ಮೂಲಕ', 'ಹೆಚ್ಚು', 'ನ', 'ಆ', 'ಕೆಲವು',
    'ಅನೇಕ', 'ಎರಡು', 'ಹಾಗು', 'ಪ್ರಮುಖ', 'ಇದನ್ನು', 'ಇದರ', 'ಸುಮಾರು', 'ಅದರ', 'ಅದು', 'ಮೊದಲ',
    'ಬಗ್ಗೆ', 'ನಲ್ಲಿ', 'ರಂದು', 'ಇತರ', 'ಅತ್ಯಂತ', 'ಹೆಚ್ಚಿನ', 'ಸಹ', 'ಸಾಮಾನ್ಯವಾಗಿ', 'ನೇ',
    'ಹಲವಾರು', 'ಹೊಸ', 'ದಿ', 'ಕಡಿಮೆ', 'ಯಾವುದೇ', 'ಹೊಂದಿದೆ', 'ದೊಡ್ಡ', 'ಅನ್ನು', 'ಇವರು',
    'ಪ್ರಕಾರ', 'ಇದೆ', 'ಮಾತ್ರ', 'ಕೂಡ', 'ಇಲ್ಲಿ', 'ಎಲ್ಲಾ', 'ವಿವಿಧ', 'ಅದನ್ನು', 'ಹಲವು', 'ರಿಂದ',
    'ಕೇವಲ', 'ದ', 'ದಕ್ಷಿಣ', 'ಗೆ', 'ಅವನ', 'ಅತಿ', 'ನೆಯ', 'ಬಹಳ', 'ಕೆಲಸ', 'ಎಲ್ಲ', 'ಪ್ರತಿ',
    'ಇತ್ಯಾದಿ', 'ಇವು', 'ಬೇರೆ', 'ಹೀಗೆ', 'ನಡುವೆ', 'ಇದಕ್ಕೆ', 'ಎಸ್', 'ಇವರ', 'ಮೊದಲು', 'ಶ್ರೀ',
    'ಮಾಡುವ', 'ಇದರಲ್ಲಿ', 'ರೀತಿಯ', 'ಮಾಡಿದ', 'ಕಾಲ', 'ಅಲ್ಲಿ', 'ಮಾಡಲು', 'ಅದೇ', 'ಈಗ', 'ಅವು',
    'ಗಳು', 'ಎ', 'ಎಂಬುದು', 'ಅವನು', 'ಅಂದರೆ', 'ಅವರಿಗೆ', 'ಇರುವ', 'ವಿಶೇಷ', 'ಮುಂದೆ', 'ಅವುಗಳ',
    'ಮುಂತಾದ', 'ಮೂಲ', 'ಬಿ', 'ಮೀ', 'ಒಂದೇ', 'ಇನ್ನೂ', 'ಹೆಚ್ಚಾಗಿ', 'ಮಾಡಿ', 'ಅವರನ್ನು', 'ಇದೇ',
    'ಯ', 'ರೀತಿಯಲ್ಲಿ', 'ಜೊತೆ', 'ಅದರಲ್ಲಿ', 'ಮಾಡಿದರು', 'ನಡೆದ', 'ಆಗ', 'ಮತ್ತೆ', 'ಪೂರ್ವ', 'ಆತ',
    'ಬಂದ', 'ಯಾವ', 'ಒಟ್ಟು', 'ಇತರೆ', 'ಹಿಂದೆ', 'ಪ್ರಮಾಣದ', 'ಗಳನ್ನು', 'ಕುರಿತು', 'ಯು', 'ಆದ್ದರಿಂದ',
    'ಅಲ್ಲದೆ', 'ನಗರದ', 'ಮೇಲಿನ', 'ಏಕೆಂದರೆ', 'ರಷ್ಟು', 'ಎಂಬುದನ್ನು', 'ಬಾರಿ', 'ಎಂದರೆ', 'ಹಿಂದಿನ',
    'ಆದರೂ', 'ಆದ', 'ಸಂಬಂಧಿಸಿದ', 'ಮತ್ತೊಂದು', 'ಸಿ', 'ಆತನ'
]


In [19]:
data = []

for index, row in df_train.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in kannada_stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})

new_df_train = pd.DataFrame(data)
new_df_train.head(30)


,text,label
0,tik tok alli jagala madtidralla adra baggenu o...,0
1,anyone from kerala here,2
2,movie rerelease madi plss,0
3,amazon prime alli bittidirayella manele nodtare,0
4,guru sure news nanu tik tok dawn lod madeda ya...,0
5,ಸುದೀಪ್ ಸರ್ ಅಂಡ್ ದರ್ಶನ್ ಸರ್ ಇರೋ ಫ್ಯಾನ್ಸ್ ಫಾಲೋಯಿ...,0
6,ade old same story,1
7,superb rakshit sir sobg superb macking,2
8,hai neel prithvi wonderful movie dia,2
9,ಏನ್ ಗುರು ನಿನ್ನ ಚರಿತ್ರೆ ಸೃಷ್ಟಿಸುವ ಅವತಾರ ತುಂಬಾ ಚ...,0


In [20]:
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('kn')

In [21]:
from langdetect import detect, LangDetectException
from tqdm import tqdm

def stem_kannada_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)

# Assuming new_df_train is your DataFrame containing text data
# Add tqdm to the loop to track progress
for index, row in tqdm(new_df_train.iterrows(), total=len(new_df_train)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'kn':
        stemmed_text = stem_kannada_text(text)
        new_df_train.at[index, 'text'] = stemmed_text


print(new_df_train.head(20))


100%|██████████| 6217/6217 [01:24<00:00, 73.91it/s]

                                                 text label
0   tik tok alli jagala madtidralla adra baggenu o...     0
1                            anyone from kerala here      2
2                          movie rerelease madi plss      0
3    amazon prime alli bittidirayella manele nodtare      0
4   guru sure news nanu tik tok dawn lod madeda ya...     0
5   ಸುದೀಪ ಸರ್ ಅಂಡ್ ದರ್ಶನ ಸರ್ ಇರೋ ಫ್ಯಾನ್ ಫಾಲೋ ರಕ್ಷಿ...     0
6                                 ade old same story      1
7             superb rakshit sir sobg superb macking      2
8               hai neel prithvi wonderful movie dia      2
9   ಏನ್ ಗುರು ನಿನ್ನ ಚರಿತ್ರೆ ಸೃಷ್ಟಿಸುವ ಅವತಾರ ತುಂಬಾ ಚ...     0
10  e tara film ge ella kaitaididdu welcooome item...     0
11               super guru smiling face with hearts      0
12                           different aagide song n      0
13                          d boos fans all the best      2
14  ನಿಜವಾಗಿಯೂ ಅದ್ಭುತ heartly heltidini plz avrigel...     0
15                                   i a

In [22]:
new_df_train['text'].isnull().sum()

0

In [23]:
new_df_train.to_csv('kannada_dataset_1_train.csv', index=False)

In [24]:
df_test = pd.DataFrame(dataset["validation"])
label_mapping = {
    0: 'Not_offensive',
    1: 'Offensive',
    2: 'Offensive',
    3: 'Offensive',
    4: 'Offensive',
    5: 'not-Kannada',
}
df_test['label'].replace(label_mapping, inplace=True)

label_mapping1 = {
    'Not_offensive':0,
    'Offensive':1,
    'not-Kannada':2,
}
df_test['label'].replace(label_mapping1, inplace=True)
print(df_test.info())
label_counts = df_test['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    777 non-null    object
 1   label   777 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.3+ KB
None
0    426
2    191
1    160
Name: label, dtype: int64


In [25]:
df_test.head(20)

,text,label
0,100 days pakka,0
1,Super sir,2
2,Hands up,2
3,Sari nivu video na roast madi adre madvaga pub...,0
4,Krishana shapa tatteleebeku,0
5,Lifestyle super guru,2
6,ಕನ್ನಡ ಇವತ್ತು ರಾಷ್ಟ್ರೀಯ ಮತ್ತು ಅಂತಾರಾಷ್ಟ್ರೀಯ ಮಟ್...,0
7,ಕೇಳಿ ಕಾದಿರುವ ಭಾಂದವರೇnಭುವಿಯಲ್ಲಿ ಅವನ ಅರಿತವರೆnಯಾರ...,0
8,sanjukuamar chatnale ğy,0
9,Hindi song release madlilva?,0


In [26]:
translator = google_translator()
for index, row in df_test.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'kn': # Text is Kannada
            for key, value in found.items():
                emojiEng = value
                # Translate to Kannada
                transText = translator.translate(emojiEng, lang_tgt='kn', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_test.at[index, 'text'] = text

In [27]:
df_test

,text,label
0,100 days pakka,0
1,Super sir,2
2,Hands up,2
3,Sari nivu video na roast madi adre madvaga pub...,0
4,Krishana shapa tatteleebeku,0
...,...,...
772,Nanu kuda direction course kalita iddene...nim...,0
773,Ninige bere kelsa ilvaa madoke,1
774,മലയാളീസ് ഹാൻഡ്‌സ് അപ്പ്‌,2
775,ದರ್ಶನ್ ಅಭಿಮಾನಿಗಳ ಕಡೆಯಿಂದ ಆಲ್ ದ ಬೆಸ್ಟ್ ಆಲ್ ದ ಬೆ...,0


In [28]:
for index, row in df_test.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [29]:
import contractions
# contracted text
for index, row in df_test.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_test.at[index, 'text'] = expanded_text


for index, row in df_test.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_test.at[index, 'text'] = Sentence.lower()

In [30]:
df_test

,text,label
0,100 days pakka,0
1,super sir,2
2,hands up,2
3,sari nivu video na roast madi adre madvaga pub...,0
4,krishana shapa tatteleebeku,0
...,...,...
772,nanu kuda direction course kalita iddenenimma ...,0
773,ninige bere kelsa ilvaa madoke,1
774,മലയാളീസ് ഹാൻഡ്‌സ് അപ്പ്‌,2
775,ದರ್ಶನ್ ಅಭಿಮಾನಿಗಳ ಕಡೆಯಿಂದ ಆಲ್ ದ ಬೆಸ್ಟ್ ಆಲ್ ದ ಬೆ...,0


In [31]:
data = []

for index, row in df_test.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in kannada_stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})

new_df_test = pd.DataFrame(data)
new_df_test.head(30)


,text,label
0,100 days pakka,0
1,super sir,2
2,hands up,2
3,sari nivu video na roast madi adre madvaga pub...,0
4,krishana shapa tatteleebeku,0
5,lifestyle super guru,2
6,ಕನ್ನಡ ಇವತ್ತು ರಾಷ್ಟ್ರೀಯ ಅಂತಾರಾಷ್ಟ್ರೀಯ ಮಟ್ಟದಲ್ಲಿ...,0
7,ಕೇಳಿ ಕಾದಿರುವ ಭಾಂದವರೇnಭುವಿಯಲ್ಲಿ ಅರಿತವರೆnಯಾರಿಲ್ಲ...,0
8,sanjukuamar chatnale ğy,0
9,hindi song release madlilva,0


In [32]:
from langdetect import detect, LangDetectException
from tqdm import tqdm

def stem_kannada_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)


for index, row in tqdm(new_df_test.iterrows(), total=len(new_df_test)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'kn':
        stemmed_text = stem_kannada_text(text)
        new_df_test.at[index, 'text'] = stemmed_text


print(new_df_test.head(20))


100%|██████████| 777/777 [00:10<00:00, 71.25it/s]

                                                 text label
0                                     100 days pakka      0
1                                          super sir      2
2                                           hands up      2
3   sari nivu video na roast madi adre madvaga pub...     0
4                        krishana shapa tatteleebeku      0
5                               lifestyle super guru      2
6   ಕನ್ನಡ ಇವತ್ತ ರಾಷ್ಟ್ರೀಯ ಅಂತಾರಾಷ್ಟ್ರೀಯ ಮಟ್ಟದಲ್ಲಿ ...     0
7   ಕೇಳಿ ಕಾದ ಭಾಂದವರೇnಭುವಿಯಲ್ಲಿ ಅರಿತವರೆnಯಾರಿಲ್ಲ ಬಿಡ...     0
8                            sanjukuamar chatnale ğy      0
9                        hindi song release madlilva      0
10                                   ಶಂಕ್ರಣ್ಣ is back     0
11                              trend setter awesome      0
12  jeevana banda haage accept madtha sagtidira wi...     0
13                         d boss abhimani kadeyinda      0
14                   make avane srimanarayana part 2      0
15                    ತಿಪ್ಪ ಕಥೆ ತುಂಬಾ ಚೆ

In [33]:
new_df_test['text'].isnull().sum()

0

In [34]:
new_df_test.to_csv('kannada_dataset_1_test.csv', index=False)